Условие: используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел.

In [2]:
!pip install pyspark

In [13]:
#Импорт библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, when, col, lit

In [5]:
#Запуск Спарк сессии
spark = SparkSession.builder.appName("Odd Sum").getOrCreate()

In [6]:
#Импорт значений
data = [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,)]

In [7]:
#создание датафрейм
df = spark.createDataFrame(data, ["rate"])

In [11]:
#расчёт суммы нечётных значений
odd_sum = df.select(sum(when(col("rate") % 2 != 0, col("rate"))).alias("odd_sum")).first().odd_sum


In [14]:
#добавляем столбец с суммой нечётных значений
df = df.withColumn("odd_sum", lit(odd_sum))

In [15]:
df.show()

+----+-------+
|rate|odd_sum|
+----+-------+
|   1|     25|
|   2|     25|
|   3|     25|
|   4|     25|
|   5|     25|
|   6|     25|
|   7|     25|
|   8|     25|
|   9|     25|
|  10|     25|
+----+-------+



In [16]:
spark.stop()